In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
import pickle
from string import punctuation

!pip install pymystem3
from pymystem3 import Mystem

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [15]:
recipes=pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv',
                   converters={'tags': lambda x: x[1:-1].replace('"', '').replace("'", "").replace(" ", "").split(',')}
                   )

recipes["Text"] = recipes["name"] + ' ' + recipes["description"]
recipes = recipes.drop(['id', 'minutes','contributor_id', 'nutrition', 'n_steps', 'n_ingredients', 'submitted', 'name', 'description', 'steps', 'ingredients'], axis=1)
recipes.head()

,tags,Text
0,"[60-minutes-or-less, time-to-make, course, mai...",arriba baked winter squash mexican style aut...
1,"[30-minutes-or-less, time-to-make, course, mai...",a bit different breakfast pizza this recipe c...
2,"[time-to-make, course, preparation, main-dish,...",all in the kitchen chili this modified versio...
3,"[60-minutes-or-less, time-to-make, course, mai...","alouette potatoes this is a super easy, great..."
4,"[weeknight, time-to-make, course, main-ingredi...",amish tomato ketchup for canning my dh's ami...


In [29]:
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text)
    text = re.sub("," , " ", text)
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    return text

recipes.Text = recipes.Text.fillna(" no recipe ")
recipes.Text = recipes.Text.apply(lambda x: clean_text(x))
recipes.Text[2]

'all in the kitchen chili this modified version of moms chili was a hit at our christmas party we made an extra large pot to have some left to freeze but it never made it to the freezer it was a favorite by all perfect for any cold and rainy day you wont find this one in a cookbook it is truly an original'

In [26]:
a = str(recipes.tags[0]).strip('[]')
a = re.sub("," , " |", a)
a = re.sub("'" , "", a)
a

'60-minutes-or-less | time-to-make | course | main-ingredient | cuisine | preparation | occasion | north-american | side-dishes | vegetables | mexican | easy | fall | holiday-event | vegetarian | winter | dietary | christmas | seasonal | squash'

In [27]:
def to_string(text):
    text = str(text).strip('[]')
    text = re.sub("," , " |", text)
    text = re.sub("'" , "", text)
    return text
    
    
recipes.tags = recipes.tags.apply(lambda x: to_string(x))
recipes.tags[2]

'time-to-make | course | preparation | main-dish | chili | crock-pot-slow-cooker | dietary | equipment | 4-hours-or-less'

In [30]:
recipes.head()

,tags,Text
0,60-minutes-or-less | time-to-make | course | m...,arriba baked winter squash mexican style autum...
1,30-minutes-or-less | time-to-make | course | m...,a bit different breakfast pizza this recipe ca...
2,time-to-make | course | preparation | main-dis...,all in the kitchen chili this modified version...
3,60-minutes-or-less | time-to-make | course | m...,alouette potatoes this is a super easy great t...
4,weeknight | time-to-make | course | main-ingre...,amish tomato ketchup for canning my dhs amish ...


In [33]:
b = recipes.head(4000)
b.to_excel('mycsvfile4.xlsx')

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
#recipes = recipes.join(pd.DataFrame(mlb.fit_transform(recipes.pop('tags')),
#                          columns=mlb.classes_,
#                          index=recipes.index))
#recipes.head()
#b = recipes.head(4000)
#b.to_csv('mycsvfile2.csv',index=False, sep='|')

In [ ]:
#mini = recipes.head(5)

In [ ]:
#recipes.Text[2].sub("\'", "") 

In [ ]:
def freq_words(x, terms = 30): 
    all_words = ' '.join([text for text in x]) 
    all_words = all_words.split() 
    fdist = nltk.FreqDist(all_words) 
    words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())}) 
  
  # selecting top 20 most frequent words 
    d = words_df.nlargest(columns="count", n = terms) 
  
  # visualize words and frequencies
    plt.figure(figsize=(12,15)) 
    ax = sns.barplot(data=d, x= "count", y = "word") 
    ax.set(ylabel = 'Word') 
    plt.show()
  
  #print 100 most frequent words 
#freq_words(recipes.clean_text, 30)

In [ ]:
mystem = Mystem() 
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token != " " \
              and token.strip() not in punctuation]    
    text = " ".join(tokens)
    return text

#recipes.clean_text = recipes.clean_text.apply(preprocess_text)

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

#recipes.clean_text = recipes.clean_text.apply(lambda x: remove_stopwords(x))

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

#recipes.clean_text = recipes.clean_text.apply(lambda x: remove_stopwords(x))

#freq_words(recipes.clean_text, 100)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
#multilabel_binarizer.fit(recipes.tags)

# transform target variable
#y = multilabel_binarizer.transform(recipes.tags)

In [ ]:
#tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [ ]:
#xtrain, xval, ytrain, yval = train_test_split(recipes.clean_text, y, test_size=0.2, random_state=9)

In [ ]:
# create TF-IDF features
#xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
#xval_tfidf = tfidf_vectorizer.transform(xval)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

In [ ]:
#lr = LogisticRegression(max_iter = 10000)
#clf = OneVsRestClassifier(lr)
# fit model on train data
#clf.fit(xtrain_tfidf, ytrain)

#y_pred = clf.predict(xval_tfidf)

In [ ]:
#y_pred[3]

In [ ]:
#multilabel_binarizer.inverse_transform(y_pred)[3]

In [ ]:
#f1_score(yval, y_pred, average="micro")

In [ ]:
def infer_tags(q):
    q = clean_text(q)
    q = preprocess_text(q)
    q = remove_stopwords(q)
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

#infer_tags('fried chicken')

In [ ]:
#recipes_mini=recipes.head(3)
#recipes_mini.head()

In [ ]:
#recipes_mini.tags

from sklearn.preprocessing import MultiLabelBinarizer

#multilabel_binarizer = MultiLabelBinarizer()
#multilabel_binarizer.fit(recipes.tags)

# transform target variable
#y = multilabel_binarizer.transform(recipes.tags)

In [ ]:
#y[1]

In [ ]:
#recipes.tags[1][2]

In [ ]:
#a = recipes.tags.explode()

In [ ]:
#a.head()

In [ ]:
#a = df.A.value_counts()
#m = recipes.tags.explode().isin(a.index[a<35])

In [ ]:
#mini = recipes.head(5)
#mini = mini.join(pd.DataFrame(mini.tags.tolist(),index=mini.index))
#mini.head()

In [ ]:
#df.drop('Col3', 1).join(df.Col3.str.join('|').str.get_dummies())

In [ ]:
#recipes.shape

In [ ]:
#recipes.head()

In [ ]:
#a.plot(kind='barh', figsize = (10, 70))

In [ ]:
def converter(x):
    return frozenset(x) if isinstance(x, (set, list)) else x

#print(len(mini.tags.apply(converter).unique()))